In [1]:
# 기본 경로 설정
# ===============================================================
import os
import subprocess

PROJECT_NAME = "rag"

ROOT_DIR = os.getcwd()

try:
    from google.colab import drive, userdata
    IS_COLAB_MODE = True
    print("코랩 모드")

except ModuleNotFoundError as e:
    IS_COLAB_MODE = False
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR, ".."))
    os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    print(f"로컬 모드")


    DATA_DIR = os.path.join(ROOT_DIR, "data")
    RAW_DIR = os.path.join(DATA_DIR, "raw")
    PDF_DIR = os.path.join(DATA_DIR, "pdf")
    os.makedirs(DATA_DIR, exist_ok=True)


# 환경변수 로드 설정
def get_secret(key_name: str):
    if IS_COLAB_MODE:
        return userdata.get(key_name)
    else:
        from dotenv import load_dotenv
        load_dotenv(dotenv_path=os.path.join(ROOT_DIR, ".env"))
        return os.getenv(key_name)


if IS_COLAB_MODE:
    drive.mount('/content/drive')

    DATA_DIR = os.path.join(ROOT_DIR, "drive", "MyDrive", "codeit-part3-team4", "data")
    RAW_DIR = os.path.join(DATA_DIR, "raw")
    PDF_DIR = os.path.join(DATA_DIR, "pdf")
    os.makedirs(DATA_DIR, exist_ok=True)

    SAVE_DIR = os.path.join(DATA_DIR, "runs", PROJECT_NAME)
    os.makedirs(SAVE_DIR, exist_ok=True)


# .DS_Store 파일 삭제
_ = subprocess.run(
    ["find", DATA_DIR, "-name", ".DS_Store", "-type", "f", "-delete"],
    check=True
)

로컬 모드


In [2]:
import preprocess_v4 as pp
from preprocess_v4 import ALL_DATA, index_pages, all_data_path

from pathlib import Path

import json
from glob import glob

# 표
from img2table.document import PDF
from img2table.ocr import TesseractOCR
import pandas as pd

# 텍스트
from langchain_community.document_loaders import PyPDFLoader

/Users/won/dev/00_codeit/0_mission/200_DL_RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 목차 데이터는 페이지 내용에서 삭제

In [3]:
# 실행 완료

# for file_name, page_info_dict in index_pages.items():
#     if file_name not in ALL_DATA.keys():
#         continue

#     index_page_label = page_info_dict["index_page_label"]

#     if not index_page_label:
#         continue

#     count = len(index_page_label)

#     for dict_ in ALL_DATA[file_name]["metadata"]:
#         if int(dict_["page_label"]) in index_page_label:
#             dict_["text"] = None
#             dict_["table"] = None
#             dict_["section"] = None
#             count -= 1
        
#         if count == 0:
#             break

# 텍스트 정제 업그레이드

In [4]:
for value in ALL_DATA.values():
    list_ = value["metadata"]

    for dict_ in list_:
        text = dict_["text"]
        if text:
            dict_["text"] = pp.clean_text(text)

In [5]:
with open(all_data_path, "w", encoding="utf-8") as f:
    json.dump(ALL_DATA, f, indent=2, ensure_ascii=False)